In [1]:
import pandas as pd
import numpy as np

from ast import literal_eval                                  # 문자열 파싱 라이브러리
from sklearn.feature_extraction.text import CountVectorizer   # 
from sklearn.metrics.pairwise import cosine_similarity        # 코사인 유사도 라이브러리

import warnings

warnings.filterwarnings('ignore')

# beer_df : CBF를 위한 파일만들기

In [ ]:
# 데이터 가져오기(맥주 데이터)
beer_load = pd.read_csv('맥주이름_특징_tag_완료_최종.csv',encoding='utf-8')
rating = pd.read_csv('맥주.csv', encoding='utf-8')

In [ ]:
# beer_rating: 맥주 종류별 평점평균
beer_rating = pd.DataFrame(columns=['맥주이름','평점평균','평가횟수'])
beer_rating['맥주이름'] = list(set(rating['맥주']))
beer_rating

# 평점평균 구하기
for beer in beer_rating['맥주이름']:
    temp = rating[beer==rating['맥주']]['평점'].mean()
    beer_rating['평점평균'][beer_rating['맥주이름']==beer]=temp
    
# 평가횟수 구하기
for beer in beer_rating['맥주이름']:
    temp = len(rating[beer==rating['맥주']])
    beer_rating['평가횟수'][beer_rating['맥주이름']==beer]=temp
    
beer_rating

In [ ]:
# 총 데이터 합병
beer_data = pd.merge(beer_load, beer_rating, on='맥주이름')
beer_data.tail(50)

In [ ]:
# 데이터 준비 및 정재 list 형태로 바꿔주기 
# beer_data에서 총 6개의 feature(Aroma, Flavor, Balance, Season, Paring  Food, Body)를 가져온다 
beer_df = pd.DataFrame(beer_data, columns = ['Aroma','Flavor','Balance','Season','Paring Food', 'Body'])


# 데이터 타입 확인: data셀 하나에 들어있는 내용들은 str
type(beer_df['Aroma'].iloc[0])
beer_df['Aroma'].iloc[0]


# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

In [ ]:
beer_df.info()

In [ ]:
beer_data.info()

## 데이터 이름 전처리 - Leffe Brune / ... -> Leffe Brune

In [ ]:
ogname = ['Sapporo Premium Beer / Draft Beer',
         'Guinness Original 4.2% (Ireland/UK)',
         'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
         'Bavaria Pilsener / Premium Beer',
         'Leffe Brune / Bruin / Brown']
name = [
    'Sapporo Premium Beer',
    'Guinness Original 4.2%',
    'Franziskaner Hefe-Weissbier',
    'Bavaria Pilsener',
    'Leffe Brune'
]

In [ ]:
# 전처리 전
beer_data.맥주이름.unique()

In [ ]:
for i in range(len(ogname)):
    beer_data['맥주이름'] = beer_data['맥주이름'].apply(lambda x : name[i] if x==ogname[i] else x)

In [ ]:
beer_data[beer_data['맥주이름'].str.contains('/')]['맥주이름'].unique()

In [ ]:
# 전처리 후

beer_data.맥주이름.unique()

In [ ]:
beer_data.reset_index(drop=True, inplace=True)

In [ ]:
beer_data

In [ ]:
beer_data

In [ ]:
beer_data.info()

In [ ]:
beer_data.to_csv('맥주_cbf_data.csv', encoding='utf-8')

# 추출한 Feature별 컨텐츠 유사도 측정

## Something에 무엇을 넣느냐에 따라 달라진다.
### 총 5개의 항목 'Aroma', 'Flavor', 'Balance', 'Paring Food', 'Body'

In [8]:
beer_data = pd.read_csv('맥주_cbf_data.csv', encoding='utf-8')
beer_df = pd.read_csv('맥주_cbf_feature.csv', encoding='utf-8')

In [9]:
# something = 'Aroma'
# something = 'Flavor'
# something = 'Balance'
# sometbeer_data = pd.read_csv('맥주_cbf_data.csv', encoding='utf-8')hing = 'Paring Food'
something = 'Body'

In [10]:
# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

In [11]:
beer_df

,Unnamed: 0,Aroma,Flavor,Balance,Season,Paring Food,Body,Body_literal
0,0,"[고소한 맥아향, 강한 향, 빵 향, 달콤한 향, 풀잎 향]","[쓴 맛, 상쾌함, 생강 힌트]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 탄산]",드라이 깔끔함 가벼움 탄산
1,1,"[고소한 맥아향, 강한 향, 쌀 향, 풀잎 향]","[쓴 맛, 상쾌함, 쌀 풍미, 약간의 단맛, 약간의 쓴 맛]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 약간의 매끄러움]",드라이 깔끔함 가벼움 약간의 매끄러움
2,2,"[고소한 맥아향, 강한 향, 희미한 옥수수향]","[쓴 맛, 상쾌함, 가벼운 맥아 풍미, 옥수수 풍미, 약간의 쓴맛]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드, 매운 음식]","[드라이, 깔끔함, 가벼움, 가벼운 탄산]",드라이 깔끔함 가벼움 가벼운 탄산
3,3,"[고소한 맥아향, 강한 향, 가벼운 스컹스 향, 풀잎 향]","[쓴 맛, 상쾌함, 스컹크 풍미]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 탄산]",드라이 깔끔함 탄산
4,4,"[고소한 맥아향, 강한 향, 가벼운 낱알 몰트 향]","[쓴 맛, 상쾌함, 맥아 풍미]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 적당한 탄산]",드라이 깔끔함 적당한 탄산
5,5,"[고소한 맥아향, 강한 향, 매운 향, 플로랄 홉향, 가벼운 향]","[쓴 맛, 상쾌함, 약간의 단맛, 맥아 풍미]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 적당한 탄산]",드라이 깔끔함 가벼움 적당한 탄산
6,6,"[고소한 맥아향, 강한 향, 약한 향, 과일향]","[쓴 맛, 상쾌함, 보조 맥아 힌트, 사과 주스 맛]",[미디엄],"[연중 내내, 더운 여름]","[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 적당한 탄산]",드라이 깔끔함 적당한 탄산
7,7,"[고소한 맥아향, 강한 향, 가벼운 향]","[쓴 맛, 상쾌함, 약간 단맛, 김치에 좋은 맥주]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 적당한 탄산]",드라이 깔끔함 가벼움 적당한 탄산
8,8,"[고소한 맥아향, 강한 향, 스파이시한 향, 약한 빵 향]","[쓴 맛, 상쾌함, 순함, 약한 쓴맛]","[미디엄, 균형]",[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 부드러움, 가벼움, 탄산]",드라이 깔끔함 부드러움 가벼움 탄산
9,9,"[고소한 맥아향, 강한 향, 가벼운 향]","[쓴 맛, 상쾌함, 약한 달콤한 맥아]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움]",드라이 깔끔함 가벼움


In [12]:
beer_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수
0,0,0,0,Kloud Original Gravity,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '빵 향', '달콤한 향', '풀잎 향']","['쓴 맛', '상쾌함', '생강 힌트']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '탄산']",2.301010,99
1,1,1,2,Asahi Super Dry,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '쌀 향', '풀잎 향']","['쓴 맛', '상쾌함', '쌀 풍미', '약간의 단맛', '약간의 쓴 맛']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '약간의 매끄러움']",2.192882,1981
2,2,2,3,Tsingtao,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '희미한 옥수수향']","['쓴 맛', '상쾌함', '가벼운 맥아 풍미', '옥수수 풍미', '약간의 쓴맛']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드', '매운 음식']","['드라이', '깔끔함', '가벼움', '가벼운 탄산']",2.058092,2212
3,3,3,4,Heineken,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '가벼운 스컹스 향', '풀잎 향']","['쓴 맛', '상쾌함', '스컹크 풍미']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '탄산']",2.159061,4133
4,4,4,5,Kirin Ichiban,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '가벼운 낱알 몰트 향']","['쓴 맛', '상쾌함', '맥아 풍미']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '적당한 탄산']",2.224910,1943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,72,72,72,Burge Meester Shandy Lemon,Flavored - Fruit,미국-크래프트 맥주,['과일향'],"['과일 풍미', '시큼한 풍미', '크리미함']","['청량함', '드라이', '약간 달콤', '일정 수준의 신맛']",['연중 내내'],"['샐러드', '가벼운 음식', '가벼운 디저트']","['라이트', '미디엄']",2.300000,2
73,73,73,68,Mallaskosken Kuohu Amber Lager Special Edition,Amber Lager - International / Vienna,라거,"['과일향', '고소한 맥아향', '약한 빵 향', '허브향']","['구운 풍미', '쓴 맛', '견과류 맥아 풍미', '가벼움', '단 맛']",['청럄함'],['연중 내내'],"['구운 붉은 고기', '바비큐 고기', '햄', '고구마 튀김', '살코기 생선'...",['가벼움'],3.133333,3
74,74,74,73,Red Rock,Amber Lager - International / Vienna,라거,"['과일향', '고소한 맥아향', '약한 빵 향', '허브향']","['구운 풍미', '쓴 맛', '견과류 맥아 풍미', '가벼움', '단 맛']",['청럄함'],['연중 내내'],"['구운 붉은 고기', '바비큐 고기', '햄', '고구마 튀김', '살코기 생선'...",['가벼움'],1.840000,15
75,75,75,74,Erdinger Weissbier Dunkel,Weissbier - Dunkelweizen,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['과일향', '스파이시한 향', '캐러멜향']","['밀크세이크 질감', '가벼운 곡물 풍미', '홉의 풍미 많지 않음', '고도의 ...","['드라이', '맥아 풍미', '곡물 풍미', '리치', '크리미']","['연중 내내', '여름']","['샐러드', '해산물', '가벼운 음식', '바이스부르스트']","['걸쭉', '드라이']",3.223954,1983


In [13]:
# CountVectorizer를 활용하기 위한 전처리 
beer_df[something +'_literal'] = beer_df[something].apply(lambda x : (' ').join(x))
beer_df[something +'_literal']

0              드라이 깔끔함 가벼움 탄산
1        드라이 깔끔함 가벼움 약간의 매끄러움
2          드라이 깔끔함 가벼움 가벼운 탄산
3                  드라이 깔끔함 탄산
4              드라이 깔끔함 적당한 탄산
5          드라이 깔끔함 가벼움 적당한 탄산
6              드라이 깔끔함 적당한 탄산
7          드라이 깔끔함 가벼움 적당한 탄산
8         드라이 깔끔함 부드러움 가벼움 탄산
9                 드라이 깔끔함 가벼움
10             드라이 깔끔함 적당한 탄산
11             드라이 깔끔함 적당한 탄산
12            드라이 깔끔함 적당한 탄산화
13                드라이 깔끔함 가벼움
14                    드라이 깔끔함
15                드라이 깔끔함 가벼움
16            드라이 깔끔함 적당한 탄산화
17                    드라이 깔끔함
18                드라이 깔끔함 가벼움
19              드라이 깔끔함 많은 탄산
20          드라이 깔끔함 가벼움 적은 탄산
21                    드라이 깔끔함
22                 드라이 깔끔함 탄산
23                    드라이 깔끔함
24                    드라이 깔끔함
25                    드라이 깔끔함
26                    드라이 깔끔함
27         드라이 깔끔함 가벼움 적당한 탄산
28         드라이 깔끔함 가벼움 풍부한 탄산
29           드라이 가벼운 탄산화 부드러움
30         드라이 미디엄 매우 크리미 크리미
31         드라이 미디엄 매우 크리미 크리미
32             미디엄 깔끔함 탄산 가벼움
33        

In [14]:
#CountVectorizer로 학습시키기
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))   # 특징 2개(ngram_range: 1<= n <=2)
something_mat = count_vect.fit_transform(beer_df[something + '_literal'])

# 77개의 맥주 중에 178개 Aroma의 'Aroma 매트릭스'가 생성됨
print(something_mat.shape)

(60, 65)


In [15]:
something_mat

<60x65 sparse matrix of type '<class 'numpy.int64'>'
	with 329 stored elements in Compressed Sparse Row format>

In [16]:
# 77개의 맥주에 대한 가각 유사한 맥주들이 계산 됨

something_sim = cosine_similarity(something_mat,something_mat)
print(something_sim.shape)
print(something_sim)

(60, 60)
[[1.         0.62994079 0.75592895 ... 0.         0.37796447 0.21821789]
 [0.62994079 1.         0.55555556 ... 0.         0.33333333 0.19245009]
 [0.75592895 0.55555556 1.         ... 0.         0.33333333 0.19245009]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.37796447 0.33333333 0.33333333 ... 0.         1.         0.        ]
 [0.21821789 0.19245009 0.19245009 ... 0.         0.         1.        ]]


In [17]:
# 순서 보기
# 숫자가 작을수록 유사도가 높은 맥주
something_sim_sorted_ind = something_sim.argsort()[:, ::-1]
print(something_sim_sorted_ind[:1])

[[ 0 18  9 13 15  2  5 28 20 27  8  7  3 22 24 23 26 17 21 14 25  1 19 11
   4  6 10 56 41 12 47 49 48 45 32 33 16 43 58 46 34 39 50 36 59 37 38 40
  44 52 53 54 42 35 29 30 31 51 55 57]]


# 코사인 유사도로 맥주 추천

In [18]:
def find_sim_beer_ver1(beer_data, sorted_ind, beer_name, top_n=10):
    
    beer_name = beer_data[beer_data['맥주이름'] == beer_name]
    beer_index = beer_name.index.values
    similar_indexes = sorted_ind[beer_index, :(top_n)]
    
    similar_inidexes = similar_indexes.reshape(-1)
    
    return beer_data.iloc[similar_inidexes]

In [19]:
beer_data.sort_values(by='맥주이름')['맥주이름'].unique()

array(['ARK Seoulite Ale', 'Asahi Super Dry', 'Asahi Super Dry Black',
       'Bali Hai Premium Lager', 'Bavaria 8.6 (Original)',
       'Bavaria Pilsener', "Beck's",
       'Berliner Kindl Jubiläums Pilsener Premium', 'Bintang Pilsener',
       'Budweiser', 'Burge Meester Shandy Lemon', 'Carlsberg Pilsner',
       'Cass Fresh', 'Castlemaine XXXX Gold Lager', 'Desperados',
       'Edelweiss Weissbier Snowfresh', 'Egger Grapefruit Radler',
       'Egger Märzenbier', 'Egger Zwickl', 'Eichbaum Apostel Bräu',
       'Erdinger Weissbier', 'Erdinger Weissbier Dunkel', 'FiLite',
       'Franziskaner Hefe-Weissbier', 'Gambrinus Originál 10° (Desítka)',
       'Grolsch Premium Lager', 'Guinness Draught',
       'Guinness Original 4.2%', 'Harbin Beer', 'Heineken',
       'Heineken Dark Lager', 'Hite D (Dry Finish)', 'Hite Extra Cold',
       'Hite Prime Max', 'Hoegaarden', 'Hoegaarden Cherry',
       'Hoegaarden Rosée', 'Holsten Premium Bier', 'Jeju Wit Ale',
       'Kirin Ichiban', 'Kloud Fitz 

In [21]:
# 10개의 맥주 추천 결과
# Pilsner Urquel로 검색한 결과, 대체로 필스터 혹은 비슷한 성질으 가진 페일에일이 추천되는 것을 알 수 있다.
beer_name='Leffe Brune'

similar_beers = find_sim_beer_ver1(beer_data, something_sim_sorted_ind, beer_name, 3)
similar_beers

IndexError: index 70 is out of bounds for axis 0 with size 60

# Real Test

In [22]:
beer_data['맥주이름'].unique()

array(['Kloud Original Gravity', 'Asahi Super Dry', 'Tsingtao',
       'Heineken', 'Kirin Ichiban', 'Sapporo Premium Beer',
       'Stella Artois', 'San Miguel', 'Cass Fresh', 'Victoria Bitter',
       'Bintang Pilsener', "Suntory The Premium Malt's", 'Tiger Beer',
       'Sapporo Yebisu', 'Peroni Nastro Azzurro',
       'Bali Hai Premium Lager', 'Holsten Premium Bier',
       'Bavaria Pilsener', 'Lapin Kulta IV A', 'Grolsch Premium Lager',
       'Castlemaine XXXX Gold Lager', 'Löwenbräu Original (4.2%)',
       'Harbin Beer', "Beck's", 'Kloud Fitz Super Clear',
       'OB Premier Pilsner', 'Hite D (Dry Finish)', 'Hite Prime Max',
       'Hite Extra Cold', 'Miller Genuine Draft (MGD)', 'Budweiser',
       'FiLite', 'Stephans Bräu Premium Lager', 'Guinness Draught',
       'Kronenbourg 1664 Blanc', 'Jeju Wit Ale', 'Platinum White Ale',
       'Pilsner Urquell', 'Gambrinus Originál 10° (Desítka)', 'Stout',
       'Krombacher Weizen', 'Reeper B. Blondes Weissbier',
       'Tsingtao Wheat

In [23]:
def find_sim_beer_ver1(beer_data, sorted_ind, beer_name, top_n=4):
    
    beer_name = beer_data[beer_data['맥주이름'] == beer_name]
    
    beer_index = beer_name.index.values
    similar_indexes = sorted_ind[beer_index, :(top_n)]
    
    similar_inidexes = similar_indexes.reshape(-1)
    
    return beer_data.iloc[similar_inidexes]

In [24]:
beer_data = pd.read_csv('맥주_cbf_data.csv', encoding='utf-8')
beer_df = pd.read_csv('맥주_cbf_feature.csv', encoding='utf-8')

beer_name = 'Leffe Brune'

# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

# something = 'Aroma'
# something = 'Flavor'
# something = 'Balance'
# something = 'Paring Food'
something = 'Body'

# CountVectorizer를 활용하기 위한 전처리
beer_df[something + '_literal'] = beer_df[something].apply(lambda x: (' ').join(x))

# CountVectorizer로 학습시키기
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))  # 특징 2개(ngram_range: 1<= n <=2)
something_mat = count_vect.fit_transform(beer_df[something + '_literal'])

# 77개의 맥주에 대한 가각 유사한 맥주들이 계산 됨
something_sim = cosine_similarity(something_mat, something_mat)

# 유사도 높은순으로 역순 배치 후 인덱스 추출
something_sim_sorted_ind = something_sim.argsort()[:, ::-1]

# 10개의 맥주 추천 결과
# Pilsner Urquel로 검색한 결과, 대체로 필스터 혹은 비슷한 성질을 가진 페일에일이 추천되는 것을 알 수 있다.

similar_beers = find_sim_beer_ver1(beer_data, something_sim_sorted_ind, beer_name, 4)

similar_beers = similar_beers[similar_beers['맥주이름'] != beer_name]

result = similar_beers['맥주이름'].tolist()

IndexError: index 70 is out of bounds for axis 0 with size 60